In [1]:
import sys
sys.path.append('../')

import meta_dataloader.TCGA
import meta_dataloader.utils
import models.mlp, models.gcn
import numpy as np
import data.gene_graphs
import collections
import sklearn.metrics
import sklearn.model_selection

%load_ext autoreload
%autoreload 2

In [2]:
tasks = meta_dataloader.TCGA.TCGAMeta(download=True)

In [5]:
task = tasks[18]
print(task.id)
print(task._samples.shape)
print(np.asarray(task._labels).shape)
print(collections.Counter(task._labels))

('Node_nature2012', 'BRCA')
(791, 16300)
(791,)
Counter({0: 385, 1: 264, 2: 99, 3: 43})


In [6]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(task._samples, 
                                                                            task._labels, 
                                                                            stratify=task._labels,
                                                                            train_size=50,
                                                                            shuffle=True,
                                                                            random_state=0
                                                                           )

/Users/ieee8023/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
model = models.mlp.MLP(name="MLP_lay2_chan512", num_layer=2, channels=32, 
                       metric=sklearn.metrics.accuracy_score)


Early stopping metric is accuracy_score


In [9]:
model.fit(X_train, y_train)

  batch (0/40)train loss:1.4485
  batch (10/40)train loss:1.7593
  batch (20/40)train loss:0.8636
  batch (30/40)train loss:2.5160
epoch: 0, time: 0.00, valid_metric: 0.50, train_metric: 0.47
  batch (0/40)train loss:1.1351
  batch (10/40)train loss:1.6471
  batch (20/40)train loss:1.4191
  batch (30/40)train loss:1.6239
epoch: 1, time: 0.00, valid_metric: 0.30, train_metric: 0.35
  batch (0/40)train loss:0.9753
  batch (10/40)train loss:1.2319
  batch (20/40)train loss:1.1354
  batch (30/40)train loss:1.4713
epoch: 2, time: 0.00, valid_metric: 0.50, train_metric: 0.47
  batch (0/40)train loss:1.3273
  batch (10/40)train loss:1.1436
  batch (20/40)train loss:1.0292
  batch (30/40)train loss:1.2251
epoch: 3, time: 0.00, valid_metric: 0.50, train_metric: 0.53
  batch (0/40)train loss:1.1108
  batch (10/40)train loss:1.0720
  batch (20/40)train loss:1.1779
  batch (30/40)train loss:1.1855
epoch: 4, time: 0.00, valid_metric: 0.50, train_metric: 0.47
  batch (0/40)train loss:1.1189
  batch 

In [10]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
print(sklearn.metrics.accuracy_score(y_test, y_pred))


0.48313090418353577


In [11]:
graph = data.gene_graphs.GeneManiaGraph()
adj = graph.adj()

Torrent name: genemania.pkl, Size: 9.61MB


In [12]:
model = models.gcn.GCN(name="GCN_lay3_chan64_emb32_dropout_agg_hierarchy", 
                       dropout=True, 
                       num_layer=4, 
                       channels=64, 
                       embedding=32, 
                       aggregation="hierarchy",
                       lr=0.0001,
                       num_epochs=5
                      )

Early stopping metric is accuracy_score


In [14]:
model.fit(X_train, y_train, adj)

/Users/ieee8023/Library/Python/3.7/lib/python/site-packages/scipy/sparse/compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


Reducing graph by a factor of 2 to 8150 nodes
Found cache for /Users/ieee8023/git/gene-graph-conv/.cache/hierarchicalb9a8a435080edb44dd7545b40248ef139ecffef54a4be93b3a89847e040373058150.npy
Reducing graph by a factor of 2 to 4075 nodes
Found cache for /Users/ieee8023/git/gene-graph-conv/.cache/hierarchicalbb9e42c8c4f9f496dd14aa64465f6321b54a97ddd7d02b7252ba9eebc32acdb14075.npy
Reducing graph by a factor of 2 to 2037 nodes
Found cache for /Users/ieee8023/git/gene-graph-conv/.cache/hierarchical9ce023dfad768dca97b2f9074706b0e7a41c70fa35014204c21d6665325cb2bb2037.npy
Reducing graph by a factor of 2 to 1018 nodes
Found cache for /Users/ieee8023/git/gene-graph-conv/.cache/hierarchicalc7e9ed6e28ab56bbbf992ab185873e96f035f1c695f8e8aba35af49c10ce58281018.npy
  batch (0/40)train loss:1.3919
  batch (10/40)train loss:1.3098
  batch (20/40)train loss:1.1913
  batch (30/40)train loss:1.2925
epoch: 0, time: 7.44, valid_metric: 0.50, train_metric: 0.47
  batch (0/40)train loss:1.1192
  batch (10/40)t

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
print(sklearn.metrics.accuracy_score(y_test, y_pred))